In [5]:
/*
 * Si Python et/ou R ne fonctionnent pas, voyez: https://github.com/dotnet/interactive/blob/HEAD/docs/jupyter-in-polyglot-notebooks.md
 *
 * Enlevez les commentaires qui suivent afin de connecter Python et R.
 */

// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3

// #!connect jupyter --kernel-name Rkernel --kernel-spec ir

In [28]:
#r "nuget: Microsoft.Extensions.Http, 8.0.0"

Installed Packages Microsoft.Extensions.Http, 8.0.0

## Assigner les valeurs d'échantillonage

Modifiez les valeurs d'échantillonage afin de modifier la taille, le nombre de records sur chaque carte, et cetera.

In [293]:
public class SampleSettings
{
  public static int minRecords = 5;

  public static int sampleSizePerTag = 2;
}

## Préparation pour l'utilisation du API de TMNF-X (Trackmania Nations Forever Exchange), ansi que de leur site normal

In [1]:

public enum Order1 {
  None = 0,
  UploadDateOldest = 1,
  UploadDateNewestDefault = 2,
  UpdateDateOldest = 3,
  UpdateDateNewest = 4,
  AwardCountAscending = 5,
  AwardCountDescending = 6,
  CommentCountAscending = 7,
  CommentCountDescending = 8,
  ActivityOldest = 9,
  ActivityNewest = 10,
  TrackNameAZ = 11,
  TrackNameZA = 12,
  AuthorNameAZ = 13,
  AuthorNameZA = 14,
  DifficultyAscending = 15,
  DifficultyDescending = 16,
  DownloadCountAscending = 17,
  DownloadCountDescending = 18,
  TrackValueAscending = 19,
  TrackValueDescending = 20,
  AwardsthisweekAscending = 21,
  AwardsthisweekDescending = 22,
  AwardsthismonthAscending = 23,
  AwardsthismonthDescending = 24,
  LastAwardDateOldest = 25,
  LastAwardDateNewest = 26,
  WorldRecordsetOldest = 27,
  WorldRecordsetNewest = 28,
  PersonalrecordsetOldestLoginrequired = 29,
  PersonalrecordsetNewestLoginrequired = 30,
  PersonalawardsetOldestLoginrequired = 31,
  PersonalawardsetNewestLoginrequired = 32,
  PersonaldownloadOldestLoginrequired = 33,
  PersonaldownloadNewestLoginrequired = 34,
  PersonalcommentsetOldestLoginrequired = 35,
  PersonalcommentsetNewestLoginrequired = 36,
  UserRecordPositionLowest = 37,
  UserRecordPositionHighest = 38,
}

public enum Tag {
  Normal = 0,
  //Stunt = 1,
  //Maze = 2,
  Offroad = 3,
  //Laps = 4,
  Fullspeed = 5,
  //LOL = 6,
  Tech = 7,
  SpeedTech = 8,
  //RPG = 9,
  PressForward = 10,
  //Trial = 11,
  Grass = 12,
}

public enum PrimaryType {
  Race = 0,
  Puzzle = 1,
  Platform = 2,
  Stunts = 3,
  Shortcut = 4,
  Laps = 5,
}

public enum LBType {
  Allreplaytypes = -1,
  Normal = 0,
  Classic = 1,
  Nadeo = 2,
  Useless = 3,
}

public enum Difficulty {
  Beginner = 0,
  Intermediate = 1,
  Expert = 2,
  Lunatic = 3,
}

public enum Routes {
  Single = 0,
  Multiple = 1,
  Symmetrical = 2,
}

In [2]:
#nullable enable

public class TMXResponses
{
  public static DateTime newestMapAt { get; set; }
  public static DateTime oldestMapAt { get; set; }

  public class ParsedResponse<T>
  {
    public Boolean More { get; set; }
    public T[]? Results { get; set; }
  }

  public class MapResponse
  {
      public int? TrackId { get; set; }
      public string? TrackName { get; set; }
      public string? UId { get; set; }
      public int? AuthorTime { get; set; }
      public Uploader? Uploader { get; set; }
      public DateTime? UploadedAt { get; set; }
      public DateTime? UpdatedAt { get; set; }
      public int? PrimaryType { get; set; } // enum
      public int? TrackValue { get; set; }
      public int? Style { get; set; } // enum
      public int? Routes { get; set; } // enum
      public int? Difficulty { get; set; }
      public int? Environment { get; set; } // enum
      public int? Car { get; set; } // enum
      public WRReplay? WRReplay { get; set; }
  }

  public class Uploader
  {
    public int? UserId { get; set; }
    public string? Name { get; set; }
  }

  public class WRReplay
  {
    public int ReplayId { get; set; }
    public int ReplayTime { get; set; }
  }

  public class ReplayResponse
  {
      public Int64? ReplayId { get; set; }
      public User? User { get; set; }
      public Int32? ReplayTime { get; set; }
      public Int32? ReplayScore { get; set; }
      public Int32? ReplayRespawns { get; set; }
      public Int32? Score { get; set; }
      public Int32? Position { get; set; }
      public Int32? IsBest { get; set; }
      public Int32? IsLeaderboard { get; set; }
      public DateTime? TrackAt { get; set; }
      public DateTime? ReplayAt { get; set; }
  }

  public class User
  {
    public int? UserId { get; set; }
    public String? Name { get; set; }
  }
}

In [26]:
#nullable enable

public class TMXInputs
{
  private static String inputParametersToString<T>(T inputParametersObject)
  {
    // Voir https://stackoverflow.com/questions/4276566/how-can-you-loop-over-the-properties-of-a-class

    StringBuilder paramaterString = new StringBuilder();

    Boolean needsAnd = false;
    foreach(var prop in typeof(T).GetProperties()) {
      object? value = prop.GetValue(inputParametersObject, null);

      if (value == null) { continue; }

      if (needsAnd) {
        paramaterString.Append("&");
      } else { needsAnd = true; }

      if (prop.Name == "fields") {
        paramaterString.Append($"fields={String.Join("%2C", (String[]) value)}");
        continue;
      }

      // If DateTime
      if (value is DateTime) {
        string dateString = ((DateTime) value).ToString("s", System.Globalization.CultureInfo.InvariantCulture);

        paramaterString.Append($"{prop.Name}={dateString}");
        continue;
      }

      // If Enum[]
      if (value.GetType().IsArray && (((Array) value).GetValue(0)??"NotEnum").GetType().IsEnum) {
        paramaterString.Append($"{prop.Name}=");

        int[] enumValue =  (int[]) value;

        paramaterString.Append(enumValue[0]);

        for (int i = 1; i < enumValue.Length; i++) {
          paramaterString.Append($"%2C{enumValue[i]}");
        }
        
        continue;
      }

      // If Enum
      if (value is Enum) {
        paramaterString.Append($"{prop.Name}={((Enum) value).ToString("D")}");
        continue;
      }

      paramaterString.Append($"{prop.Name}={value}");
    }
    
    return paramaterString.ToString();
  }

  public class MapInputsParameters {
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Search Order
      /// </summary>
    public Order1? order1 { get; set; }
      /// <summary>
      /// Amount of results (default: 40)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified TrackId and given order - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified TrackId and given order - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified TrackId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// Filter by TrackId's
      /// </summary>
    public Int64[,]? id { get; set; }
      /// <summary>
      /// Filter by UId or Track file signatures
      /// </summary>
    public String[,]? uid { get; set; }
      /// <summary>
      /// Filter by Tags[]
      /// </summary>
    public Tag[]? tag { get; set; }
      /// <summary>
      /// Filter by excluding Tags[]
      /// </summary>
    public Tag[]? etag { get; set; }
      /// <summary>
      /// true: track must have all specified tags, false: track must have one of the specified tags
      /// </summary>
    public Boolean taginclusive { get; } = true;
      /// <summary>
      /// Filter by PrimaryType
      /// </summary>
    public PrimaryType primarytype { get; } = PrimaryType.Race;
      /// <summary>
      /// Filter by min. UploadedAt
      /// </summary>
    public DateTime? uploadedafter { get; set; }
      /// <summary>
      /// Filter by max. UploadedAt
      /// </summary>
    public DateTime? uploadedbefore { get; set; }
      /// <summary>
      /// Filter by min. AuthorTime in ms
      /// </summary>
    public Int32? authortimemin { get; } = 30000;
      /// <summary>
      /// Filter by max. AuthorTime in ms
      /// </summary>
    public Int32? authortimemax { get; } = 120000;
      /// <summary>
      /// Filter by Routes
      /// </summary>
    public Routes routes { get; } = Routes.Single;
      /// <summary>
      /// 1: Track has at least one replay
      /// </summary>
    public Int32 inhasrecord { get; } = 1;
      /// <summary>
      /// 1: Track is environment mixed (Environment =/= Car)
      /// </summary>
    public Int32 inenvmix { get; } = 0;
      /// <summary>
      /// 1: Track requires to be played with TMUnlimiter, 0: vanilla game
      /// </summary>
    public Int32 inunlimiter { get; } = 0;

    public override string ToString(){
      return inputParametersToString<MapInputsParameters>(this);
    }  
  }

  public class ReplayInputsParameters {
      /// <summary>
      /// TM1X TrackId of track containing the replays
      /// </summary>
    public String? trackId { get; set; }
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Amount of results (default: 10)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified ReplayId - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified ReplayId - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified ReplayId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// 1: Returns only the best replay
      /// </summary>
    public Int32? best { get; set; } = 1;
      /// <summary>
      /// Filter by driver TM1X UserId
      /// </summary>
    public Int64? userid { get; set; }

    public override string ToString(){
      return inputParametersToString<ReplayInputsParameters>(this);
    }  
  }
}

In [29]:
using System;
using System.Globalization;
using System.Net.Http;
using System.Net;
using System.Text.Json;

#nullable enable

public class TMXAPI
{
  public static HttpClient sharedClient = new()
  {
      BaseAddress = new Uri("https://tmnf.exchange"),
  };

  // private static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponse(TMXInputs.MapInputsParameters paramaters)
  // {
  //   using HttpResponseMessage response = await sharedClient.GetAsync($"api/tracks?{paramaters}"); // Note: If "parameters" can be chosen by the user, add sanitization

  //   Console.WriteLine($"Get: api/tracks?{paramaters}");
    
  //   response.EnsureSuccessStatusCode();
    
  //   var jsonResponse = await response.Content.ReadAsStringAsync();

  //   TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

  //   return mapResponse;
  // }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponseFromID(String[] fields, String id)
  {
    String relativeURL = $"api/tracks?fields={String.Join("%2C", fields)}&id={id}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine(relativeURL);
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

    return mapResponse;
  }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>?> getTopReplaysResponseFromMapID(String[] fields, String trackId)
  {
    TMXInputs.ReplayInputsParameters inputsParameters = new TMXInputs.ReplayInputsParameters{ trackId = trackId, fields = fields, count = SampleSettings.minRecords};

    String relativeURL = $"api/replays?{inputsParameters}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine(relativeURL);
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>? replayResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>>(jsonResponse);

    return replayResponse;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L200
  private static Uri? GetRequestUriFromResponse(HttpResponseMessage response)
  {
      if (response.RequestMessage is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request message. This is odd...");
          return null;
      }

      if (response.RequestMessage.RequestUri is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request URI. This is odd...");
          return null;
      }

      return response.RequestMessage.RequestUri;
  }

  // Voir https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L217
  private static string? GetTrackIdFromUri(Uri uri)
  {
      var trackId = uri.Segments.LastOrDefault();

      if (trackId is null) { return null; }

      return trackId;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs
  public static async Task<string?> getRandomMapID(TMXInputs.MapInputsParameters paramaters)
  {
    string? trackId = null;
    
    using var response = await sharedClient.SendAsync(new HttpRequestMessage(HttpMethod.Head, $"trackrandom?{paramaters}")); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine($"Head: trackrandom?{paramaters}");

    if (response.StatusCode == HttpStatusCode.NotFound)
    {
      // I have written something incorrectly

      throw new Exception("No map meets the filter; the programmer has written erronous parameters");
    }
    
    response.EnsureSuccessStatusCode();
    
    var requestUri = GetRequestUriFromResponse(response);

    if (requestUri is null)
    {
        Console.Error.WriteLine("RequestUri of /trackrandom is null");
        return null;
    }

    trackId = GetTrackIdFromUri(requestUri);

    return trackId;
  }
}


## Sélection aléatoire stratifié de circuits et de _replays_

Pour sélectionner un circuit aléatoire, nous utilisons le sous-répertoire `/trackrandom` du site TMNF-X. Les `minRecords` _replays_ les plus rapides -- de joueurs différents -- sont ensuite cherchés. Si un nombre de _replays_ équivalent à `minRecords` sont trouvés, les identifiants du circuit et du record mondial sont enregistrés.

In [92]:

public class RandomMap()
{
  public static Dictionary<Tag, List<String[]>> chosenMaps = (Enum.GetValues<Tag>()).ToDictionary(key => key, value => new List<String[]>());

  public static async Task selectMapIds() {
    foreach (Tag tag in Enum.GetValues<Tag>()) {
      Console.WriteLine("_______________________________________________________________________________________________________________");
      string chosenId = null;

      int remainingSamples = SampleSettings.sampleSizePerTag;
      for (int i = SampleSettings.sampleSizePerTag * 20; i > 0 && remainingSamples > 0; i--) {
        Console.WriteLine("------");
        chosenId = await TMXAPI.getRandomMapID(new TMXInputs.MapInputsParameters{tag = [tag]});

        TMXResponses.ParsedResponse<TMXResponses.MapResponse> mapResponse = await TMXAPI.getMapResponseFromID(["TrackValue"], chosenId);

        Console.WriteLine($"Queried map id: {chosenId}");

        TMXResponses.ParsedResponse<TMXResponses.ReplayResponse> replaysResponse = await TMXAPI.getTopReplaysResponseFromMapID(["ReplayId","User.Name"], chosenId);

        if(replaysResponse.Results.Length < SampleSettings.minRecords) { continue; }
        remainingSamples--;

        replaysResponse.Display();

        Console.WriteLine($"Fastest replay id: {replaysResponse.Results[0].ReplayId}");

        chosenMaps[tag].Add(new String[]{chosenId, replaysResponse.Results[0].ReplayId.ToString()});
      }
    }
  }
}

await RandomMap.selectMapIds();

RandomMap.chosenMaps

_______________________________________________________________________________________________________________
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=7452296
Queried map id: 7452296
api/replays?trackId=7452296&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=6734846
Queried map id: 6734846
api/replays?trackId=6734846&fields=ReplayId%2CUser.Name&count=5&best=1


Submission#5+TMXResponses+ParsedResponse`1[Submission#5+TMXResponses+ReplayResponse] More True Results index value 0 Submission#5+TMXResponses+ReplayResponse ReplayId 10697578 User Submission#5+TMXResponses+User UserId <null> Name Law.Axion ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 1 Submission#5+TMXResponses+ReplayResponse ReplayId 9531956 User Submission#5+TMXResponses+User UserId <null> Name |PT| CakeyFA ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 2 Submission#5+TMXResponses+ReplayResponse ReplayId 10330532 User Submission#5+TMXResponses+User UserId <null> Name Klony420 ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 3 Submission#5+TMXResponses+ReplayResponse ReplayId 6858397 User Submission#5+TMXResponses+User UserId <null> Name Orion.esu ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 4 Submission#5+TMXResponses+ReplayResponse ReplayId 6798629 User Submission#5+TMXResponses+User UserId <null> Name DogBolt ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null>

Fastest replay id: 10697578
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=958577
Queried map id: 958577
api/replays?trackId=958577&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=224770
Queried map id: 224770
api/replays?trackId=224770&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=413382
Queried map id: 413382
api/replays?trackId=413382&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=0&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasre

Submission#5+TMXResponses+ParsedResponse`1[Submission#5+TMXResponses+ReplayResponse] More True Results index value 0 Submission#5+TMXResponses+ReplayResponse ReplayId 6148415 User Submission#5+TMXResponses+User UserId <null> Name LeG.Torny ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 1 Submission#5+TMXResponses+ReplayResponse ReplayId 5821559 User Submission#5+TMXResponses+User UserId <null> Name eGow // KnockOut ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 2 Submission#5+TMXResponses+ReplayResponse ReplayId 10815000 User Submission#5+TMXResponses+User UserId <null> Name Ciekma ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 3 Submission#5+TMXResponses+ReplayResponse ReplayId 6575676 User Submission#5+TMXResponses+User UserId <null> Name zico.qtm ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 4 Submission#5+TMXResponses+ReplayResponse ReplayId 7878790 User Submission#5+TMXResponses+User UserId <null> Name the_tracker__ ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null>

Fastest replay id: 6148415
_______________________________________________________________________________________________________________
------
Head: trackrandom?tag=3&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=6392256
Queried map id: 6392256
api/replays?trackId=6392256&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=3&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=8973152
Queried map id: 8973152
api/replays?trackId=8973152&fields=ReplayId%2CUser.Name&count=5&best=1


...

Submission#5+TMXResponses+ParsedResponse`1[Submission#5+TMXResponses+ReplayResponse] More False Results index value 0 Submission#5+TMXResponses+ReplayResponse ReplayId 11608095 User Submission#5+TMXResponses+User UserId <null> Name _Donk_ ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 1 Submission#5+TMXResponses+ReplayResponse ReplayId 11600120 User Submission#5+TMXResponses+User UserId <null> Name Dalebotgaming ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 2 Submission#5+TMXResponses+ReplayResponse ReplayId 11600111 User Submission#5+TMXResponses+User UserId <null> Name czechiowo ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 3 Submission#5+TMXResponses+ReplayResponse ReplayId 11599844 User Submission#5+TMXResponses+User UserId <null> Name skroeb ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 4 Submission#5+TMXResponses+ReplayResponse ReplayId 11830315 User Submission#5+TMXResponses+User UserId <null> Name KamenKuba ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null>

Fastest replay id: 11608095
------
Head: trackrandom?tag=12&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=9687101
Queried map id: 9687101
api/replays?trackId=9687101&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=12&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=9794738
Queried map id: 9794738
api/replays?trackId=9794738&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=12&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=1&inenvmix=0&inunlimiter=0
api/tracks?fields=TrackValue&id=11872871
Queried map id: 11872871
api/replays?trackId=11872871&fields=ReplayId%2CUser.Name&count=5&best=1
------
Head: trackrandom?tag=12&taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&

Submission#5+TMXResponses+ParsedResponse`1[Submission#5+TMXResponses+ReplayResponse] More True Results index value 0 Submission#5+TMXResponses+ReplayResponse ReplayId 10933898 User Submission#5+TMXResponses+User UserId <null> Name ujogebujo ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 1 Submission#5+TMXResponses+ReplayResponse ReplayId 10933225 User Submission#5+TMXResponses+User UserId <null> Name Weregoose ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 2 Submission#5+TMXResponses+ReplayResponse ReplayId 11057612 User Submission#5+TMXResponses+User UserId <null> Name shortz.esu ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 3 Submission#5+TMXResponses+ReplayResponse ReplayId 11020532 User Submission#5+TMXResponses+User UserId <null> Name TX-Jer27 ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null> 4 Submission#5+TMXResponses+ReplayResponse ReplayId 10941642 User Submission#5+TMXResponses+User UserId <null> Name Owly ReplayTime <null> ReplayScore <null> ReplayRespawns <null> Score <null> Position <null> IsBest <null> IsLeaderboard <null> TrackAt <null> ReplayAt <null>

Fastest replay id: 10933898


key value Normal index value 0 [ 6734846, 10697578 ] 1 [ 5707987, 6148415 ] Offroad index value 0 [ 8973152, 8979745 ] 1 [ 9945663, 9964151 ] Fullspeed index value 0 [ 4248020, 4259623 ] 1 [ 10488535, 11105886 ] Tech index value 0 [ 3204309, 7945811 ] 1 [ 3711864, 3726566 ] SpeedTech index value 0 [ 5198734, 5212090 ] 1 [ 2850120, 3823286 ] PressForward index value 0 [ 5243735, 5366877 ] 1 [ 5561802, 5662558 ] Grass index value 0 [ 11599207, 11608095 ] 1 [ 10902043, 10933898 ]